In [1]:
import iris

In [2]:
month = '01'
day = '01'
run = '00'
prefix = 'prods_op_mogreps-g_2016{}{}_{}'.format(month, day, run)
prefix

'prods_op_mogreps-g_20160101_00'

In [9]:
from boto.s3.connection import S3Connection
import os

os.environ['S3_USE_SIGV4'] = 'True'

def list_keys(bucket, prefix, local_path='/usr/local/share/notebooks/data/mogreps-g/'):
    conn = S3Connection(host='s3.eu-west-2.amazonaws.com')
    bucket = conn.get_bucket(bucket)
    print(type(bucket))
    results = []
    keys = list(iter(bucket.list(prefix=prefix)))
    return keys


in_keys = list_keys('mogreps-g', prefix)
print(len(in_keys))
in_keys[:10]

<class 'boto.s3.bucket.Bucket'>
696


[<Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_003.nc>,
 <Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_006.nc>,
 <Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_009.nc>,
 <Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_012.nc>,
 <Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_015.nc>,
 <Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_018.nc>,
 <Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_021.nc>,
 <Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_024.nc>,
 <Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_027.nc>,
 <Key: mogreps-g,prods_op_mogreps-g_20160101_00_00_030.nc>]

In [39]:
import iris
import os

from iris.cube import CubeList
from dask import delayed
import dask.bag as db
import boto

def load_cubes(address):
    def add_realization(cube, field, filename):
        if not cube.coords('realization'):
            realization = int(filename.split('_')[-2])
            realization_coord = iris.coords.AuxCoord(realization, standard_name='realization', var_name='realization')
            cube.add_aux_coord(realization_coord)
        cube.coord('realization').points.dtype = 'int32'
        cube.coord('time').var_name = 'time'
        cube.coord('forecast_period').var_name = 'forecast_period'
    return iris.load(address, callback=add_realization)


def download(key_name, bucket):
    key = boto.s3.key.Key(bucket=bucket, name=key_name)
    try:
        os.mkdir('./downloads/')
    except FileExistsError:
        pass
    f_name = './downloads/{}'.format(key.name)
    key.get_contents_to_filename(f_name)
    return f_name

conn = S3Connection(host='s3.eu-west-2.amazonaws.com')
bucket = conn.get_bucket('mogreps-g')

fs = [download(key.name, bucket) for key in in_keys]

OSError: [Errno 28] No space left on device

In [ ]:
fs

In [ ]:
iris.save(cube, target=('./visibility_in_air.nc'))

In [1]:
import iris

In [2]:
!ls -lah ./visibility_in_air.nc

-rw-r--r-- 1 root root 1.3G May  3 15:03 ./visibility_in_air.nc


In [3]:
%%time
one_param = iris.load('./visibility_in_air.nc')

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 29.3 ms


/opt/conda/lib/python3.5/site-packages/iris/fileformats/cf.py:1140: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


In [4]:
!ls -lah ./prods_op_mogreps-g_20160101_00_00_006.nc

-rw-r--r-- 1 root root 35M Apr  4 14:16 ./prods_op_mogreps-g_20160101_00_00_006.nc


In [5]:
%%time
multi_param = iris.load('./prods_op_mogreps-g_20160101_00_00_006.nc')

/opt/conda/lib/python3.5/site-packages/iris/fileformats/cf.py:1140: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


CPU times: user 244 ms, sys: 4 ms, total: 248 ms
Wall time: 247 ms


Single parameter cube:
  - one parameter
  - 58 timesteps
  - twelve members

In [6]:
one_param

[<iris 'Cube' of visibility_in_air / (m) (realization: 12; time: 58; latitude: 600; longitude: 800)>]

Multi param cube:
  - 28 parameters
  - 1 timestep
  - 1 member

6 parameters have multiple pressure levels, I have not measured the impact of this

In [7]:
multi_param

[<iris 'Cube' of wet_bulb_freezing_level_altitude / (m) (latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of air_pressure_at_sea_level / (Pa) (latitude: 600; longitude: 800)>,
<iris 'Cube' of air_temperature / (K) (latitude: 600; longitude: 800)>,
<iris 'Cube' of air_temperature / (K) (pressure: 16; latitude: 600; longitude: 800)>,
<iris 'Cube' of air_temperature / (K) (latitude: 600; longitude: 800)>,
<iris 'Cube' of air_temperature / (K) (latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (latitude: 600; longitude: 800)>,
<iris 'Cube' of fog_area_fraction / (1) (latitude: 600; longitude: 800)>,
<iris 'Cube' of geopotential_height / (m) (pressure: 9; latitude: 600; longitude: 800)>,
<iris 'Cube' of high_type_cloud_area_fraction / (1) (latitude: 600; longitude: 800)>,
<iris 'Cube' of low_type_cloud_area_fraction / (1) (latitude: 600; longitude: 800)>,
<iris 'Cube' 

Time to load entire model, one parameter at a time:

In [8]:
load_one_param_ms = 20
number_of_params = 28

print('{} seconds'.format((load_one_param_ms * number_of_params) / 1000))

0.56 seconds


Time to load entire model, one timestep + member at a time:

In [9]:
load_one_timestep_ms = 250
number_of_timesteps = 58
number_of_members = 12

print('{} seconds'.format((load_one_timestep_ms * number_of_timesteps * number_of_members) / 1000))

174.0 seconds


In [ ]:
1300 * 28

In [ ]:
50 * 58 * 12